In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nasa-battery-dataset/cleaned_dataset/metadata.csv
/kaggle/input/nasa-battery-dataset/cleaned_dataset/extra_infos/README_49_50_51_52.txt
/kaggle/input/nasa-battery-dataset/cleaned_dataset/extra_infos/README_45_46_47_48.txt
/kaggle/input/nasa-battery-dataset/cleaned_dataset/extra_infos/README_05_06_07_18.txt
/kaggle/input/nasa-battery-dataset/cleaned_dataset/extra_infos/README_29_30_31_32.txt
/kaggle/input/nasa-battery-dataset/cleaned_dataset/extra_infos/README_25_26_27_28.txt
/kaggle/input/nasa-battery-dataset/cleaned_dataset/extra_infos/README_33_34_36.txt
/kaggle/input/nasa-battery-dataset/cleaned_dataset/extra_infos/README_53_54_55_56.txt
/kaggle/input/nasa-battery-dataset/cleaned_dataset/extra_infos/README_41_42_43_44.txt
/kaggle/input/nasa-battery-dataset/cleaned_dataset/extra_infos/README_38_39_40.txt
/kaggle/input/nasa-battery-dataset/cleaned_dataset/data/02421.csv
/kaggle/input/nasa-battery-dataset/cleaned_dataset/data/04530.csv
/kaggle/input/nasa-battery-dataset/c

In [2]:
import pandas as pd
import ast
from datetime import datetime
import numpy as np
import plotly.graph_objects as go
import plotly.io as pio

# Set renderer to work in Kaggle notebooks
pio.renderers.default = 'iframe_connected'

# Load data
metadata_df = pd.read_csv("/kaggle/input/nasa-battery-dataset/cleaned_dataset/metadata.csv")
file_df = metadata_df[metadata_df['type'] == 'impedance'][['start_time', 'filename', 'battery_id']]

# Filter for impedance rows and select necessary columns
impedance_df = metadata_df[metadata_df['type'] == 'impedance'][['start_time', 'Re', 'Rct', 'battery_id']]

# Function to parse start_time
def parse_start_time(value):
    try:
        if isinstance(value, str):
            value = value.strip("[]").replace(",", "")  # Remove brackets and commas
            components = [float(x) for x in value.split()]  # Split and convert to float
            if len(components) == 6:
                year, month, day, hour, minute = map(int, components[:5])
                second = int(components[5])  # Handle fractional seconds
                return datetime(year, month, day, hour, minute, second)
        elif isinstance(value, (list, np.ndarray)) and len(value) == 6:
            year, month, day, hour, minute = map(int, value[:5])
            second = int(float(value[5]))  # Handle fractional seconds
            return datetime(year, month, day, hour, minute, second)
    except (ValueError, SyntaxError, TypeError) as e:
        print(f"Failed to parse: {value}, Error: {e}")
    return pd.NaT

# Apply parsing function
impedance_df['start_time'] = impedance_df['start_time'].apply(parse_start_time)

# Drop rows with invalid start_time
impedance_df = impedance_df.dropna(subset=['start_time'])

# Sort data by start_time
impedance_df = impedance_df.sort_values(by='start_time')

# Plot for each battery_id
for battery_id in impedance_df['battery_id'].unique():
    # Filter data for the current battery_id
    battery_data = impedance_df[impedance_df['battery_id'] == battery_id]
    
    # Debugging: Print data for the current battery_id
    print(f"Data for battery_id {battery_id}:\n", battery_data.head())
    
    # Skip if no data for the current battery_id
    if battery_data.empty:
        print(f"No data available for battery_id {battery_id}")
        continue
    
    # Create the figure
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=battery_data['start_time'], 
        y=battery_data['Re'], 
        mode='lines+markers', 
        name='Re',
        line=dict(color='blue')
    ))
    fig.add_trace(go.Scatter(
        x=battery_data['start_time'], 
        y=battery_data['Rct'], 
        mode='lines+markers', 
        name='Rct',
        line=dict(color='red')
    ))
    
    # Add labels and title
    fig.update_layout(
        title=f"Impedance Data for Battery ID: {battery_id}",
        xaxis_title="Start Time",
        yaxis_title="Resistance (Ohms)",
        xaxis=dict(tickangle=45),
        legend_title="Legend",
        template="plotly"
    )
    
    # Display the plot
    fig.show()

Data for battery_id B0005:
               start_time                    Re                  Rct battery_id
5160 2008-04-18 20:55:29   0.04466870036616091  0.06945627304536996      B0005
5162 2008-04-18 22:39:16   0.04668700162486933  0.07627474098530587      B0005
5164 2008-04-19 02:14:27  0.044843430573346096   0.0679720560130687      B0005
5166 2008-04-19 03:57:24   0.04619468955099375  0.07453388918437964      B0005
5168 2008-04-19 07:32:33   0.04510114851143001  0.06852832896000328      B0005


Data for battery_id B0007:
               start_time                    Re                  Rct battery_id
5776 2008-04-18 20:55:29   0.03816813609946085  0.06158094574229446      B0007
5778 2008-04-18 22:39:16  0.042352200255114755  0.07946205744298855      B0007
5780 2008-04-19 02:14:27   0.03971312292013592  0.06180654339682964      B0007
5782 2008-04-19 03:57:24   0.04171928239305599  0.08085099253522705      B0007
5784 2008-04-19 07:32:33   0.03840146399106517  0.06100130384680727      B0007


Data for battery_id B0006:
               start_time                   Re                  Rct battery_id
4544 2008-04-18 20:55:29  0.06123359021000344   0.0785415394665875      B0006
4546 2008-04-18 22:39:16  0.06561684946388616   0.0886832084583472      B0006
4548 2008-04-19 02:14:27  0.06062991544979002  0.07802409378290423      B0006
4550 2008-04-19 03:57:24   0.0626250732462495  0.08533725531470857      B0006
4552 2008-04-19 07:32:33   0.0608598716542857  0.07840240203025303      B0006


Data for battery_id B0018:
               start_time                   Re                  Rct battery_id
6353 2008-07-07 14:43:14  0.06515815158455215  0.09555369570022001      B0018
6355 2008-07-07 16:14:05  0.06319812376707153  0.10229511275627602      B0018
6357 2008-07-07 21:21:23  0.06269644718508349  0.09161902925465781      B0018
6359 2008-07-07 22:51:31   0.0626070139256824   0.0946846935443713      B0018
6361 2008-07-08 02:24:25  0.06137843058604763  0.09188288246238666      B0018


Data for battery_id B0028:
               start_time                   Re                  Rct battery_id
1449 2009-02-13 19:03:52  0.08141136501428463  0.13435294061149733      B0028
1451 2009-02-13 22:40:45  0.08176458673559299  0.10682087072021014      B0028
1461 2009-02-14 20:28:47   0.0808258053141002    0.121797260168767      B0028
1463 2009-02-15 00:05:09  0.08049755262582456  0.10171699274990839      B0028
1465 2009-02-18 12:41:09  0.07999808984375359  0.11499739391717384      B0028


...

Data for battery_id B0051:
               start_time                                         Re  \
4443 2010-08-23 18:25:01                         0.0713694865779506   
4445 2010-08-23 22:01:52                         0.0638844452297328   
4453 2010-08-26 12:25:47  (0.08640994631573766-0.0371087352306254j)   
4455 2010-08-26 16:02:42                        0.07931821233586298   
4465 2010-08-27 09:22:52                        0.08661647240225276   

                                            Rct battery_id  
4443                        0.21835138725756187      B0051  
4445                        0.16505526574215693      B0051  
4453  (0.08640994631573766+0.0371087352306254j)      B0051  
4455                        0.18509231412826796      B0051  
4465                        0.15593299545391415      B0051  


Data for battery_id B0054:
               start_time                   Re                  Rct battery_id
6809 2010-09-03 13:00:14  0.10729220819696772   0.2218699476933641      B0054
6811 2010-09-03 16:37:04   0.1021829807494312   0.2042419605630439      B0054
6821 2010-09-04 09:14:17  0.10311804563383167  0.18810644142533564      B0054
6823 2010-09-04 12:51:09  0.10285471210914307  0.19972368912498348      B0054
6833 2010-09-05 05:10:52  0.10198137428960918   0.1821024264313652      B0054


Data for battery_id B0056:
               start_time                   Re                  Rct battery_id
7062 2010-09-03 13:00:14  0.10885981803966972  0.24235190183691696      B0056
7064 2010-09-03 16:37:04  0.10496405150737945  0.22081508118084606      B0056
7074 2010-09-04 09:14:17  0.10371268081103303  0.19982947996228057      B0056
7076 2010-09-04 12:51:09  0.10256257569474589  0.19482175015040312      B0056
7086 2010-09-05 05:10:52  0.10408988253209463   0.1949176796556943      B0056


Data for battery_id B0055:
               start_time                   Re                  Rct battery_id
7314 2010-09-03 13:00:14  0.10157829311159554  0.23471686484653742      B0055
7316 2010-09-03 16:37:04  0.09619085839657462  0.20176718938616142      B0055
7326 2010-09-04 09:14:17  0.09915026406799798  0.18329577020277954      B0055
7328 2010-09-04 12:51:09   0.0959763432768074   0.1840796258694923      B0055
7338 2010-09-05 05:10:52  0.09736948957359812  0.17386450438338188      B0055


Data for battery_id B0053:
               start_time                   Re                  Rct battery_id
6672 2010-09-17 16:02:32  0.10090192584083808  0.15478049323493148      B0053
6674 2010-09-17 19:39:33  0.09969808294894725  0.16789267957289597      B0053
6684 2010-09-18 11:56:16  0.10017921050279893   0.1499614197531393      B0053
6686 2010-09-18 15:33:19  0.09972116257585717   0.1685832323391912      B0053
6696 2010-09-19 07:46:55  0.10164308300959472  0.15569309460591851      B0053


In [3]:
import pandas as pd
import plotly.graph_objects as go
from datetime import datetime

# Assuming 'file_df' is your dataframe with start_time, filename, and battery_id
# Example: file_df = pd.DataFrame({'start_time': [...], 'filename': [...], 'battery_id': [...]})

# Initialize an empty dictionary to store the impedance data for each battery
battery_data = {}

# Iterate through each row of file_df to process each file
for index, row in file_df.iterrows():
    battery_id = row['battery_id']
    filename = row['filename']
    start_time = row['start_time']
    
    # Read the CSV file using the filename
    file_path = f"/kaggle/input/nasa-battery-dataset/cleaned_dataset/data/{filename}"  # Adjust the path as necessary
    try:
        data = pd.read_csv(file_path)
        
        # Check if 'battery_impedance' column exists in the file
        if 'Battery_impedance' in data.columns:
            # Extract real part of the complex impedance and calculate the average
            # Here, we assume the Battery_impedance column contains string representations of complex numbers
            impedance_values = data['Battery_impedance'].apply(lambda x: complex(x).real)
            avg_impedance = impedance_values.mean()

            # Store the average impedance in the battery_data dictionary
            if battery_id not in battery_data:
                battery_data[battery_id] = {'start_time': [], 'impedance': []}
            
            battery_data[battery_id]['start_time'].append(start_time)
            battery_data[battery_id]['impedance'].append(avg_impedance)
        else:
            print(f"'Battery_impedance' column not found in {filename}")
    except Exception as e:
        print(f"Error processing file {filename}: {e}")

# Plot impedance change over time for each battery_id
for battery_id, data in battery_data.items():
    # Create a plot for the current battery_id
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=data['start_time'], y=data['impedance'], mode='lines+markers', name=f'Battery {battery_id}'))
    
    # Update layout for better readability
    fig.update_layout(
        title=f'Impedance Change Over Time for Battery {battery_id}',
        xaxis_title='Start Time',
        yaxis_title='Impedance (Ohms)',
        xaxis=dict(tickangle=45),
        template='plotly_white'
    )
    
    # Display the plot
    fig.show()